## 반드시 처음부터 끝까지 스켈레톤 코드를 살펴보고 구현하기 시작하길 바란다

## 1. 스켈레톤 코드를 [복사 및 편집] 하여 사용한다.
## 2. 아래의 [Empty Module 3개]를 직접 구현한다.
## (필수) 코드 작성 전에 Overview의 Description을 읽고, 본 프로젝트의 방향성을 이해하고 시작하세요.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 본 프로젝트를 위한 패키지 로드
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# 자연어 처리를 위한 라이브러리 다운로드
- nltk 라이브러리에서 punkt 데이터를 다운 받음, 이 데이터는 영화 리뷰와 같은 문서 데이터로 문자의 tokeninizer를 위해서 필요하다
- nltk 라이브러리를 이용해서 불용어를 다운 받음

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 데이터 로드

In [4]:
df_train = pd.read_csv("../input/2022-ml-project2/train.csv",encoding="latin-1")
df_test = pd.read_csv("../input/2022-ml-project2/test.csv",encoding="latin-1")

In [5]:
df_train.head()

,Unnamed: 0,Data,Class
0,0,No I'm in the same boat. Still here at my moms...,ham
1,1,(Bank of Granite issues Strong-Buy) EXPLOSIVE ...,spam
2,2,They r giving a second chance to rahul dengra.,ham
3,3,O i played smash bros &lt;#&gt; religiously.,ham
4,4,PRIVATE! Your 2003 Account Statement for 07973...,spam


In [6]:
X_train = df_train["Data"]
y_train = df_train["Class"]
X_test = df_test["Data"]

# [Empty Module #1] 텍스트 데이터 전처리  

목표: 텍스트 데이터를 처리하기 위한 여러 과정을 거쳐, 머신을 위한 데이터를 만든다. 

```
[input]
--------------
- text: 텍스트 문장 데이터 

[output]
--------------
- text: 전처리를 완료한 문장 데이터 
    
```

In [7]:
# ------------------------------------------------
# [Empty Module #1] 텍스트 데이터 전처리
# ------------------------------------------------
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1] re.sub 사용해 text 속 '[^A-Za-z]' 외의 문자만을 찾아내 제거한후, pre_words 변수에 저장
#      1) pattern은 '[^A-Za-z]', repl=' ' 로 각각 설정.
#      2) 이모지나 숫자,점과 같은 문자외의 것들을 제거했다. (이모지는 감정 분석과 관련해서 몇가지 의미를 나타내지만 이 테스크에서는 이러한 의미도 제거함.)
#
# [2] pre_words의 lower 내장 함수를 이용해 대문자들은 소문자로 변경
#      1)  대, 소문자가 구분되어 있으면 "Go"와 "go" 와 같이 동일한 단어를 머신은 다른 단어로 취급한다. 따라서 대문자를 모두 소문자로 변경.
#
# [3] word_tokenize 함수를 이용해 pre_word 를 토큰화하여 word를 리스트화한 후 tokenized_words변수에 저장
#
# [4] nltk 라이브러리로 다운 받은 stopwords의 "words" 내장 함수를 이용해 english 불용어를 찾아서 stops 변수에 저장  
#      1) 불용어: 텍스트 분류에서 불용어는 텍스트의 중요도을 결정하는데 영향을 미치지 않는 단어임. 
#                    (ex: the, we, a , will), 따라서 불필요한 단어가 예측 모델에 악영향을 끼칠 수 있기 때문에 제거.
#
# [5] [3] 에서 찾은 문자열 중 단어가 [4] 에서 찾은 불용어 속에 없을 경우, tokenized_words_remove 리스트에 append 
#
# [6] tokenized_words_remove의 단어를 PorterStemmer 속 stem 내장 함수를 이용해, 동일 의미를 갖는 단어를 동일한 단어로 변경하는 과정을 거친 후 다시 저장.
#    
# ------------------------------------------------------------

from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

def data_processing(text):
    
    # [1] re.sub 사용해 text 속 '[^A-Za-z]' 외의 문자만을 찾아내 제거한후, pre_words 변수에 저장
    pre_words = re.sub('[^A-Za-z]', ' ', text)
    
    # [2] pre_words의 lower 내장 함수를 이용해 대문자들은 소문자로 변경
    pre_words = pre_words.lower()
    
    # [3] word_tokenize 함수를 이용해 pre_word 를 토큰화하여 word를 리스트화한 후 tokenized_words변수에 저장
    tokenized_words = word_tokenize(pre_words)
    
    # [4] nltk 라이브러리로 다운 받은 stopwords의 "words" 내장 함수를 이용해 english 불용어를 찾아서 stops 변수에 저장  
    stops = stopwords.words()
    
    tokenized_words_remove=[]
    for w in tokenized_words: 
        # [5] [3] 에서 찾은 문자열 중 단어가 [4] 에서 찾은 불용어 속에 없을 경우, tokenized_words_remove 리스트에 append 
        if w not in stops: 
            tokenized_words_remove.append(w)
            
    stemmer = PorterStemmer()
    for i in range(len(tokenized_words_remove)):
        # [6] tokenized_words_remove의 단어를 PorterStemmer 속 stem 내장 함수를 이용해, 동일 의미를 갖는 단어를 동일한 단어로 변경하는 과정을 거친 후 다시 저장
        tokenized_words_remove[i] = stemmer.stem(tokenized_words_remove[i])

    return ( " ".join( tokenized_words_remove ))

In [8]:
X_train = [data_processing(i) for i in X_train]
X_test = [data_processing(i) for i in X_test]

# [Empty Module #2] Bag of Word 

목표: 문장 데이터를 머신을 학습하기 위한 실수화된 Feature로 변경하기로한다. 

- train 과 test 데이터 전부 type을 ('U')로 변경하여 Countvectorizer를 사용한다. 
- [설명 링크](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer)

In [9]:
# ------------------------------------------------
# [Empty Module #2] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1] CountVectorizer를 정의
#           1) max_features를 100으로 지정 
#
# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
#
# [3] CountVectorizer를 이용해 X_train은 학습 및 변환(fit_transform)을 하고, X_test는 변환(transform)을 진행. 
# ------------------------------------------------

# CountVectorizer

# [1] CountVectorizer를 정의
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 100)

# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
X_train = np.array(X_train).astype("U")
X_test = np.array(X_test).astype("U")

# [3] CountVectorizer를 이용해 X_train은 학습 및 변환(fit_transform)을 하고, X_test는 변환(transform)을 진행
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [10]:
# HashingVectorizer

# [1] HashingVectorizer 정의
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer()

# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
X_train = np.array(X_train).astype("U")
X_test = np.array(X_test).astype("U")

# [3] HashingVectorizer 이용해 X_train은 학습 및 변환(fit_transform)을 하고, X_test는 변환(transform)을 진행
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)


In [11]:
# TfidfVectorizer

# [1] TfidfVectorizer 정의
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm='l1', sublinear_tf=True, use_idf=False, max_features = 7000)

# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
X_train = np.array(X_train).astype("U")
X_test = np.array(X_test).astype("U")

# [3] TfidfVectorizer 이용해 X_train은 학습 및 변환(fit_transform)을 하고, X_test는 변환(transform)을 진행
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [12]:
# ham의 경우 0으로 지정하고, spam의 경우에는 1로 라벨을 변경해줌
y_train[y_train=="ham"] = 0
y_train[y_train=="spam"] = 1
y_train = y_train.astype("uint8")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## [Empty Module #3] SVM: classifier
목표: SVC() 를 활용해 classification 을 진행

- fit()으로 train 에 대한 머신러닝 학습
- predict()으로 test 에 대한 정답을 추론 하여 반환

In [13]:

# ------------------------------------------------
# [Empty Module #3] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  SVC 선언 (베이스라인 에서 gamma="auto" 사용 )
#
# [2] X_train_features과 y_train으로 SVC 학습진행 후, X_test_features로 predict 진행
#
# [3] y_pred에 predict한 결과값 저장
# ------------------------------------------------------------

from sklearn.svm import SVC

# [1] SVC 선언 (베이스라인 에서 gamma="auto" 사용 )
svc=SVC(C = 1000)

# [2] X_train_features과 y_train으로 SVC 학습진행 후, X_test_features로 predict 진행
svc.fit(X_train_features, y_train)

# [3] y_pred에 predict한 결과값 저장
y_pred= svc.predict(X_test_features)

# Predict to CSV

In [14]:
df_pred={"ID": range(np.array(y_pred).shape[0]),"Class":y_pred}
df_pred=pd.DataFrame(df_pred)
df_pred.to_csv("submission.csv",index=False)